<a href="https://colab.research.google.com/github/lindswang/cis545finalproject/blob/main/Data_Cleaning_vFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports & Loading In Data

In [2]:
!pip install dask[dataframe]

     |████████████████████████████████| 132 kB 5.3 MB/s 


In [3]:
import datetime
import matplotlib.pyplot as plt
import multiprocessing
import numpy as np
import pandas as pd
import scipy.stats as ss
import seaborn as sns
import statsmodels.api as sm
import dask.dataframe as dd

pd.set_option('display.max_columns', None)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
# Load-in Data

from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1IvgBHiR7u_A3m5dIvdK86MuXKjO-JwRe',
                                    dest_path='/content/directory.csv')
gdd.download_file_from_google_drive(file_id='1YCZ8NWxyIgeaB1FBT9QdUzfE-NaRmwlT',
                                    dest_path='/content/sat.csv')
gdd.download_file_from_google_drive(file_id='1WJ2TKwtlADN5v-B-3KApGeNlUYZhFG9Z',
                                    dest_path='/content/state_test.csv')



raw_ccd_directory_df = pd.read_csv('directory.csv', dtype=str)
sat_act_df = pd.read_csv('sat.csv', dtype=str)
state_test_df = dd.read_csv('state_test.csv', dtype=str)

### Cleaning CCD Directory Data

#### Deal with explicit NAs

In [6]:
# Only select 2017 Data
print("Number of rows before: " +  str(raw_ccd_directory_df.shape[0]))
ccd_directory_df = raw_ccd_directory_df[raw_ccd_directory_df['year'] == '2017'].copy('deep')
print("Number of rows after: " +  str(raw_ccd_directory_df.shape[0]))

Number of rows before: 3279903
Number of rows after: 3279903


In [7]:
# Confirm that there is only one row per school
ccd_directory_df.ncessch.is_unique

True

In [8]:
# Check how many NAs there are
ccd_directory_df.isna().sum()

year                                0
ncessch                             0
ncessch_num                         0
school_id                           0
school_name                         0
leaid                               0
lea_name                            0
state_leaid                         0
seasch                              0
street_mailing                      0
city_mailing                        0
state_mailing                       0
zip_mailing                         0
street_location                     0
city_location                       0
state_location                      0
zip_location                        0
phone                               0
fips                                0
latitude                            0
longitude                           0
csa                                 0
cbsa                                0
urban_centric_locale                0
county_code                         0
school_level                        0
school_type 

In [9]:
# Notice '2438' in a bunch of places
# Seems like the same schools with a lot of NAs
# Let's examine them

ccd_directory_df[ccd_directory_df.lunch_program.isna()]['school_status'].value_counts()

# Note the following encoding:
# 2 = closed, 6 = inactive, 7 = future

# I think it's safe to drop these rows, as most of these schools are closed or inactive with no useful information
print("Number of rows before: " +  str(ccd_directory_df.shape[0]))
ccd_directory_df.dropna(subset = ['lunch_program'],inplace=True)
print("Number of rows after: " +  str(ccd_directory_df.shape[0]))

Number of rows before: 102337
Number of rows after: 99899


In [10]:
# Let's drop these two columns because it seems 'free_or_reduced_price_lunch' is a better overall indicator
ccd_directory_df.drop(columns = ['free_lunch','reduced_price_lunch'], inplace=True)

In [11]:
# Drop rows without enrollment data. It is too crucial.
# Note that this also takes care of all the NAs in lowest and highest grade offered

print("Number of rows before: " +  str(ccd_directory_df.shape[0]))
ccd_directory_df.dropna(subset = ['enrollment'], inplace=True)
print("Number of rows after: " +  str(ccd_directory_df.shape[0]))

Number of rows before: 99899
Number of rows after: 98511


In [12]:
# Drop rows without data on the number of teachers. 
# The information is too crucial to be missing

print("Number of rows before: " +  str(ccd_directory_df.shape[0]))
ccd_directory_df.dropna(subset = ['teachers_fte'], inplace=True)
print("Number of rows after: " +  str(ccd_directory_df.shape[0]))

Number of rows before: 98511
Number of rows after: 94800


In [13]:
# We're missing 70% of data for this column
# Best to just drop this column
print(ccd_directory_df.direct_certification.isna().sum()/ccd_directory_df.shape[0])
ccd_directory_df.drop(columns = ['direct_certification'],inplace=True)

0.7020780590717299


In [14]:
# Let's drop title_i_schoolwide because it seems there are two other columns describe roughly the same thing
ccd_directory_df.title_i_schoolwide.isna().sum()/ccd_directory_df.shape[0]
ccd_directory_df.drop(columns = ['title_i_schoolwide'],inplace=True)

In [15]:
# Note that this column has no useful data
# Drop the column
print((ccd_directory_df.bureau_indian_education == '1').sum())
print((ccd_directory_df.bureau_indian_education == '0').sum())
ccd_directory_df.drop(columns = ['bureau_indian_education'], inplace=True)

0
0


In [16]:
# Assume that if a school is missing data on whether it's a magnet or charter school, then they're not either
ccd_directory_df['magnet'][ccd_directory_df.magnet == '-1'] = 0
ccd_directory_df['magnet'][ccd_directory_df.magnet == '-2'] = 0
ccd_directory_df['charter'][ccd_directory_df.charter == '-1'] = 0
ccd_directory_df['charter'][ccd_directory_df.charter == '-2'] = 0

In [17]:
# Decided to drop CSA and CBSA because of all the missing values
ccd_directory_df.drop(columns = ['csa', 'cbsa'], inplace=True)

In [18]:
# Checked that no more 'closed' schools are in the dataset
assert (ccd_directory_df.school_status == '2').sum() == 0, 'Checked that no more closed schools are in the dataset'

In [19]:
# Size of the directory information after all the explicit NAs have been dropped
ccd_directory_df.shape

(94800, 45)

#### Examine rows and columns with special values (-1, -2, -3) (Missing, NA, Suppressed)

In [20]:
# Note that there's no suppressed data
(ccd_directory_df == '-3').sum() 

year                           0
ncessch                        0
ncessch_num                    0
school_id                      0
school_name                    0
leaid                          0
lea_name                       0
state_leaid                    0
seasch                         0
street_mailing                 0
city_mailing                   0
state_mailing                  0
zip_mailing                    0
street_location                0
city_location                  0
state_location                 0
zip_location                   0
phone                          0
fips                           0
latitude                       0
longitude                      0
urban_centric_locale           0
county_code                    0
school_level                   0
school_type                    0
school_status                  0
lowest_grade_offered           0
highest_grade_offered          0
title_i_status                 0
title_i_eligible               0
charter   

In [21]:
# Check how much data is missing
# Note that for columns 'lowest_grade_offered' & 'highest_grade_offered', -1 just means pre-k
(ccd_directory_df == '-1').sum() 

year                               0
ncessch                            0
ncessch_num                        0
school_id                          0
school_name                        0
leaid                              0
lea_name                           0
state_leaid                        0
seasch                             0
street_mailing                     0
city_mailing                       0
state_mailing                      0
zip_mailing                        0
street_location                    0
city_location                      0
state_location                     0
zip_location                       0
phone                              0
fips                               0
latitude                           0
longitude                          0
urban_centric_locale               0
county_code                        0
school_level                       0
school_type                        0
school_status                      0
lowest_grade_offered           30571
h

In [22]:
ccd_directory_df.virtual.value_counts()
# For the virtual column, assume if they didn't report whether they're virtual, that they're not virtual
ccd_directory_df[ccd_directory_df.virtual == '-1'] = 0

In [23]:
# Drop shared_time column, because it just means it's either a vocational or technical school
ccd_directory_df.drop(columns = 'shared_time', inplace=True)

In [24]:
# Size of the clean directory information
ccd_directory_df.shape

(94800, 44)

### Cleaning SAT ACT Data

In [ ]:
# Examine the number of NAs in each column
sat_act_df.isna().sum()

crdc_id                  0
year                     0
ncessch              29220
leaid                29220
fips                     0
sex                      0
race                     0
disability               0
lep                      0
students_SAT_ACT    620070
dtype: int64

In [26]:
# Drop schools missing ncessch ids. 
# We need the ncessch ID to join on, so it's crucial information we can't lack
sat_act_df.dropna(subset = ['ncessch'], inplace=True)

In [27]:
# Note that there are about 30 rows per school
sat_act_df.shape[0] / sat_act_df.ncessch.nunique()

30.027648624299722

In [28]:
# Each row would show a different subsection of demographics via 3-tuples i.e. sex, race, disability (1,1,99) means white, male total
sat_act_df.head()

,crdc_id,year,ncessch,leaid,fips,sex,race,disability,lep,students_SAT_ACT
0,010000201705,2017,010000201705,0100002,1,1,1,99,99,7
1,010000201705,2017,010000201705,0100002,1,1,2,99,99,11
2,010000201705,2017,010000201705,0100002,1,1,3,99,99,0
3,010000201705,2017,010000201705,0100002,1,1,4,99,99,0
4,010000201705,2017,010000201705,0100002,1,1,5,99,99,0


In [29]:
# See this link to see what each of the codes in the demographic columns mean:
# https://educationdata.urban.org/documentation/schools.html#crdc-sat-and-act-participation-by-race-and-sex

# No 'unknown' demographic data
(sat_act_df[['sex','race','disability','lep']] == '9').sum()

# Very little data is missing
(sat_act_df == '-1').sum()

# But a lot of data is NA
(sat_act_df == '-2').sum()

crdc_id                   0
year                      0
ncessch                   0
leaid                     0
fips                      0
sex                       0
race                      0
disability                0
lep                       0
students_SAT_ACT    1370620
dtype: int64

In [30]:
# Drop data where it's -2 = 'Not Applicable' 
# I believe this just means they don't have anyone of that demographic "slice"
sat_act_df = sat_act_df[sat_act_df.students_SAT_ACT != '-2'].copy('deep')

In [31]:
# It's just 7 schools with '-1' reported, i.e. Missing
sat_act_df[sat_act_df.students_SAT_ACT == '-1'].ncessch.value_counts()

361233000997    20
361233000996    20
362091004740    20
250678002272    20
250774001213    20
250678000987    20
250936001504    20
Name: ncessch, dtype: int64

In [32]:
# All of these schools reported either no data, or said no students took the SAT/ACT
sat_act_df[(sat_act_df.ncessch == '361233000996')|(sat_act_df.ncessch == '250678000987')|(sat_act_df.ncessch == '250678002272')|
          (sat_act_df.ncessch == '250936001504')|(sat_act_df.ncessch == '250774001213')|(sat_act_df.ncessch == '361233000997')|
          (sat_act_df.ncessch == '362091004740')].students_SAT_ACT.value_counts()



-1    140
0       7
Name: students_SAT_ACT, dtype: int64

In [33]:
# Upon further inspection, we find that the "totals" across all demographics for these schools were 0
# Solution should just be to drop the rows with -1 data.
sat_act_df[(sat_act_df.ncessch == '361233000996')|(sat_act_df.ncessch == '250678000987')|(sat_act_df.ncessch == '250678002272')|
          (sat_act_df.ncessch == '250936001504')|(sat_act_df.ncessch == '250774001213')|(sat_act_df.ncessch == '361233000997')|
          (sat_act_df.ncessch == '362091004740')][sat_act_df.students_SAT_ACT == '0']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


,crdc_id,year,ncessch,leaid,fips,sex,race,disability,lep,students_SAT_ACT
1213799,250678000987,2017,250678000987,2506780,25,99,99,99,99,0
1213949,250678002272,2017,250678002272,2506780,25,99,99,99,99,0
1219109,250774001213,2017,250774001213,2507740,25,99,99,99,99,0
1225739,250936001504,2017,250936001504,2509360,25,99,99,99,99,0
1749089,361233000996,2017,361233000996,3612330,36,99,99,99,99,0
1749119,361233000997,2017,361233000997,3612330,36,99,99,99,99,0
1822079,362091004740,2017,362091004740,3620910,36,99,99,99,99,0


In [34]:
# Drop aforementioned rows
sat_act_df = sat_act_df[sat_act_df.students_SAT_ACT != '-1'].copy('deep')

#### Converting row-based data into column-based data

In [35]:
# Need to consolidate these rows into 1 row per school
sat_act_df.sex.nunique() * sat_act_df.race.nunique() * sat_act_df.disability.nunique() * sat_act_df.lep.nunique()

# There are 96 possible combinations of this demographic information

96

In [36]:
# Create a column that aggregates all this demographic information 
sat_act_df['sex_race_disability_lep'] = sat_act_df['sex'].astype(str) +'_'+sat_act_df['race'].astype(str) +'_'+sat_act_df['disability'].astype(str) +'_'+sat_act_df['lep'].astype(str)

In [37]:
# Note there are actually only 30 combinations of demographic info
sat_act_df.sex_race_disability_lep.nunique() 

30

In [38]:
# That's because the data only provides intersectional data across two demographic 'axes'
sat_act_df.sex_race_disability_lep.value_counts().sort_index()

1_1_99_99      28120
1_2_99_99      28120
1_3_99_99      28120
1_4_99_99      28120
1_5_99_99      28120
1_6_99_99      28120
1_7_99_99      28120
1_99_1_99      28120
1_99_99_1      28120
1_99_99_99     28120
2_1_99_99      28120
2_2_99_99      28120
2_3_99_99      28120
2_4_99_99      28120
2_5_99_99      28120
2_6_99_99      28120
2_7_99_99      28120
2_99_1_99      28120
2_99_99_1      28120
2_99_99_99     28120
99_1_99_99     96658
99_2_99_99     96658
99_3_99_99     96658
99_4_99_99     96658
99_5_99_99     96658
99_6_99_99     96658
99_7_99_99     96658
99_99_1_99     96658
99_99_99_1     96658
99_99_99_99    96658
Name: sex_race_disability_lep, dtype: int64

In [39]:
# Get dummies for each type of demographic "slice"
sat_act_df = pd.get_dummies(sat_act_df, columns = ['sex_race_disability_lep'])

In [40]:
# Inspect the dataframe
sat_act_df.head(5)

,crdc_id,year,ncessch,leaid,fips,sex,race,disability,lep,students_SAT_ACT,sex_race_disability_lep_1_1_99_99,sex_race_disability_lep_1_2_99_99,sex_race_disability_lep_1_3_99_99,sex_race_disability_lep_1_4_99_99,sex_race_disability_lep_1_5_99_99,sex_race_disability_lep_1_6_99_99,sex_race_disability_lep_1_7_99_99,sex_race_disability_lep_1_99_1_99,sex_race_disability_lep_1_99_99_1,sex_race_disability_lep_1_99_99_99,sex_race_disability_lep_2_1_99_99,sex_race_disability_lep_2_2_99_99,sex_race_disability_lep_2_3_99_99,sex_race_disability_lep_2_4_99_99,sex_race_disability_lep_2_5_99_99,sex_race_disability_lep_2_6_99_99,sex_race_disability_lep_2_7_99_99,sex_race_disability_lep_2_99_1_99,sex_race_disability_lep_2_99_99_1,sex_race_disability_lep_2_99_99_99,sex_race_disability_lep_99_1_99_99,sex_race_disability_lep_99_2_99_99,sex_race_disability_lep_99_3_99_99,sex_race_disability_lep_99_4_99_99,sex_race_disability_lep_99_5_99_99,sex_race_disability_lep_99_6_99_99,sex_race_disability_lep_99_7_99_99,sex_race_disability_lep_99_99_1_99,sex_race_disability_lep_99_99_99_1,sex_race_disability_lep_99_99_99_99
0,010000201705,2017,010000201705,0100002,1,1,1,99,99,7,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,010000201705,2017,010000201705,0100002,1,1,2,99,99,11,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,010000201705,2017,010000201705,0100002,1,1,3,99,99,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,010000201705,2017,010000201705,0100002,1,1,4,99,99,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,010000201705,2017,010000201705,0100002,1,1,5,99,99,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [41]:
# Convert type of this column to float 
sat_act_df.students_SAT_ACT = sat_act_df.students_SAT_ACT.astype(float)

In [42]:
# Create a list of all the dummy columns
sat_act_dummy_cols =  [c for c in sat_act_df.columns if 'sex_race_disability_lep' in c]

In [43]:
# Multiply the value into the appropriate columns so that it can later be aggregated

for c in sat_act_dummy_cols:
    sat_act_df[c] = np.where(sat_act_df[c] == 1, sat_act_df[c] * sat_act_df.students_SAT_ACT, 0)

In [44]:
# Inspect dataframe
sat_act_df.head()

,crdc_id,year,ncessch,leaid,fips,sex,race,disability,lep,students_SAT_ACT,sex_race_disability_lep_1_1_99_99,sex_race_disability_lep_1_2_99_99,sex_race_disability_lep_1_3_99_99,sex_race_disability_lep_1_4_99_99,sex_race_disability_lep_1_5_99_99,sex_race_disability_lep_1_6_99_99,sex_race_disability_lep_1_7_99_99,sex_race_disability_lep_1_99_1_99,sex_race_disability_lep_1_99_99_1,sex_race_disability_lep_1_99_99_99,sex_race_disability_lep_2_1_99_99,sex_race_disability_lep_2_2_99_99,sex_race_disability_lep_2_3_99_99,sex_race_disability_lep_2_4_99_99,sex_race_disability_lep_2_5_99_99,sex_race_disability_lep_2_6_99_99,sex_race_disability_lep_2_7_99_99,sex_race_disability_lep_2_99_1_99,sex_race_disability_lep_2_99_99_1,sex_race_disability_lep_2_99_99_99,sex_race_disability_lep_99_1_99_99,sex_race_disability_lep_99_2_99_99,sex_race_disability_lep_99_3_99_99,sex_race_disability_lep_99_4_99_99,sex_race_disability_lep_99_5_99_99,sex_race_disability_lep_99_6_99_99,sex_race_disability_lep_99_7_99_99,sex_race_disability_lep_99_99_1_99,sex_race_disability_lep_99_99_99_1,sex_race_disability_lep_99_99_99_99
0,010000201705,2017,010000201705,0100002,1,1,1,99,99,7.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,010000201705,2017,010000201705,0100002,1,1,2,99,99,11.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,010000201705,2017,010000201705,0100002,1,1,3,99,99,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,010000201705,2017,010000201705,0100002,1,1,4,99,99,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,010000201705,2017,010000201705,0100002,1,1,5,99,99,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
# Aggregating down to rows

# Create a dictionary with instructions for how to aggregate in a groupby
# For all the rows with the same ncessch, sum up values in the dummy columns and take the first 'fips' code
agg_dict = dict(zip(sat_act_dummy_cols,['sum']*len(sat_act_dummy_cols)))
agg_dict['fips'] = 'first'
agg_dict['crdc_id'] = 'first'

# Do the groupby and aggregation
grouped_sat_act_df = sat_act_df.groupby('ncessch').agg(agg_dict)
grouped_sat_act_df.reset_index(inplace=True)

In [46]:
# Check that the code worked
grouped_sat_act_df.head()

,ncessch,sex_race_disability_lep_1_1_99_99,sex_race_disability_lep_1_2_99_99,sex_race_disability_lep_1_3_99_99,sex_race_disability_lep_1_4_99_99,sex_race_disability_lep_1_5_99_99,sex_race_disability_lep_1_6_99_99,sex_race_disability_lep_1_7_99_99,sex_race_disability_lep_1_99_1_99,sex_race_disability_lep_1_99_99_1,sex_race_disability_lep_1_99_99_99,sex_race_disability_lep_2_1_99_99,sex_race_disability_lep_2_2_99_99,sex_race_disability_lep_2_3_99_99,sex_race_disability_lep_2_4_99_99,sex_race_disability_lep_2_5_99_99,sex_race_disability_lep_2_6_99_99,sex_race_disability_lep_2_7_99_99,sex_race_disability_lep_2_99_1_99,sex_race_disability_lep_2_99_99_1,sex_race_disability_lep_2_99_99_99,sex_race_disability_lep_99_1_99_99,sex_race_disability_lep_99_2_99_99,sex_race_disability_lep_99_3_99_99,sex_race_disability_lep_99_4_99_99,sex_race_disability_lep_99_5_99_99,sex_race_disability_lep_99_6_99_99,sex_race_disability_lep_99_7_99_99,sex_race_disability_lep_99_99_1_99,sex_race_disability_lep_99_99_99_1,sex_race_disability_lep_99_99_99_99,fips,crdc_id
0,010000201705,7.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,1,010000201705
1,010000201706,6.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,1,010000201706
2,010000201876,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1,010000201876
3,010000500870,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,010000500870
4,010000500871,94.0,5.0,72.0,3.0,1.0,0.0,2.0,20.0,29.0,177.0,98.0,8.0,77.0,2.0,1.0,0.0,0.0,15.0,20.0,186.0,192.0,13.0,149.0,5.0,2.0,0.0,2.0,35.0,49.0,363.0,1,010000500871


#### Check that the aggregation was done correctly by verifying that the data is internally consistent

In [47]:
# Create helpful list of column names 

sex_race_cols = [
                'sex_race_disability_lep_1_1_99_99',
                'sex_race_disability_lep_1_2_99_99',
                'sex_race_disability_lep_1_3_99_99',
                'sex_race_disability_lep_1_4_99_99',
                'sex_race_disability_lep_1_5_99_99',
                'sex_race_disability_lep_1_6_99_99',
                'sex_race_disability_lep_1_7_99_99',
                'sex_race_disability_lep_2_1_99_99',
                'sex_race_disability_lep_2_2_99_99',
                'sex_race_disability_lep_2_3_99_99',
                'sex_race_disability_lep_2_4_99_99',
                'sex_race_disability_lep_2_5_99_99',
                'sex_race_disability_lep_2_6_99_99',
                'sex_race_disability_lep_2_7_99_99']

sex_disability_cols = [
                'sex_race_disability_lep_1_99_1_99',
                'sex_race_disability_lep_2_99_1_99']

sex_lep_cols = [
                'sex_race_disability_lep_1_99_99_1',
                'sex_race_disability_lep_2_99_99_1']

sex_cols = [
           'sex_race_disability_lep_1_99_99_99',
           'sex_race_disability_lep_2_99_99_99']

race_cols = [
        'sex_race_disability_lep_99_1_99_99',
       'sex_race_disability_lep_99_2_99_99',
       'sex_race_disability_lep_99_3_99_99',
       'sex_race_disability_lep_99_4_99_99',
       'sex_race_disability_lep_99_5_99_99',
       'sex_race_disability_lep_99_6_99_99',
       'sex_race_disability_lep_99_7_99_99']

disability_col = 'sex_race_disability_lep_99_99_1_99'

lep_col = 'sex_race_disability_lep_99_99_99_1'

tot_col = 'sex_race_disability_lep_99_99_99_99'

# Check all columns were captured
assert len(sex_race_cols)+len(sex_disability_cols)+len(sex_lep_cols)+len(sex_cols)+len(race_cols)+3 == 30 # the 3 refers to the disability_col,lep_col, and tot_col

In [48]:
# Verify that the data is internally consistent

# Need to read data into dask to avoid crashing kernel
dask_sat_act_df = dd.from_pandas(grouped_sat_act_df, npartitions = 3)

assert (dask_sat_act_df[sex_race_cols].sum(axis=1) == grouped_sat_act_df[tot_col]).all().compute()
assert (dask_sat_act_df[sex_disability_cols].sum(axis=1) == grouped_sat_act_df[disability_col]).all().compute()
assert (dask_sat_act_df[sex_lep_cols].sum(axis=1) == grouped_sat_act_df[lep_col]).all().compute()
assert (dask_sat_act_df[sex_cols].sum(axis=1) == grouped_sat_act_df[tot_col]).all().compute()
assert (dask_sat_act_df[race_cols].sum(axis=1) == grouped_sat_act_df[tot_col]).all().compute()

In [49]:
# It seems like all the NA values "took care of themselves", 
# We implicitly set them to 0 when performing the groupby
# And the numbers are still internally consistent

assert dask_sat_act_df.isna().sum().sum().compute() == 0

### Cleaning State Assessments Data

In [50]:
# The state_test_df has 26 columns and 4,069,382 rows
print(len(state_test_df))
print("Num unique schools:" +str(state_test_df.ncessch.nunique().compute()))
# We're averaging 45 rows per school
# We will need to convert row-based data into column-based data 
state_test_df.head()

4069382
Num unique schools:89036


,ncessch_num,ncessch,year,school_name,leaid_num,leaid,lea_name,fips,grade_edfacts,race,sex,lep,homeless,migrant,disability,econ_disadvantaged,foster_care,military_connected,read_test_num_valid,read_test_pct_prof_low,read_test_pct_prof_high,read_test_pct_prof_midpt,math_test_num_valid,math_test_pct_prof_low,math_test_pct_prof_high,math_test_pct_prof_midpt
0,10000500870,010000500870,2017,Albertville Middle School,100005,0100005,Albertville City,1,99,99,99,99,99,99,99,99,99,99,801,43,43,43.0,802,47,47,47.0
1,10000500870,010000500870,2017,Albertville Middle School,100005,0100005,Albertville City,1,99,4,99,99,99,99,99,99,99,99,5,-3,-3,-3.0,5,-3,-3,-3.0
2,10000500870,010000500870,2017,Albertville Middle School,100005,0100005,Albertville City,1,99,2,99,99,99,99,99,99,99,99,32,30,39,34.5,32,30,39,34.5
3,10000500870,010000500870,2017,Albertville Middle School,100005,0100005,Albertville City,1,99,99,99,99,99,99,1,99,99,99,51,11,19,15.0,51,11,19,15.0
4,10000500870,010000500870,2017,Albertville Middle School,100005,0100005,Albertville City,1,99,99,99,99,99,99,99,1,99,99,309,30,30,30.0,310,37,37,37.0


In [51]:
# Check that all the data is for the year 2017
state_test_df.year.value_counts().compute()

2017    4069382
Name: year, dtype: int64

In [52]:
# Check for NAs
# Note that about 3% of data has NAs
state_test_df.isna().sum().compute()

ncessch_num                      0
ncessch                          0
year                             0
school_name                      0
leaid_num                        0
leaid                            0
lea_name                         0
fips                             0
grade_edfacts                    0
race                             0
sex                              0
lep                              0
homeless                         0
migrant                          0
disability                       0
econ_disadvantaged               0
foster_care                      0
military_connected               0
read_test_num_valid          50302
read_test_pct_prof_low       50302
read_test_pct_prof_high      50302
read_test_pct_prof_midpt     50302
math_test_num_valid         106775
math_test_pct_prof_low      106775
math_test_pct_prof_high     106775
math_test_pct_prof_midpt    106775
dtype: int64

In [53]:
# Drop all NAs. 
# Hopefully, it's like the SAT ACT data where the missing values "resolve" themselves
state_test_df = state_test_df.dropna().compute()

In [54]:
# Drop some demographic slices we will choose not to use
# For the purposes of this project, we will ignore whether the students are 
# homeless, migrants, disabled, are in foster care, or are military connected
# This ensures that there are not an overwhelming number of columns
state_test_df = state_test_df[state_test_df.homeless == '99']
state_test_df = state_test_df[state_test_df.migrant == '99']
state_test_df = state_test_df[state_test_df.disability == '99']
state_test_df = state_test_df[state_test_df.foster_care == '99']
state_test_df = state_test_df[state_test_df.military_connected == '99']
print(state_test_df.shape)

(3097839, 26)


In [55]:
# Note that the datasource suppresses data when that demographic "slice" is too small (i.e. 1~5) for privacy reasons 
state_test_df[state_test_df.read_test_pct_prof_low == '-3'].read_test_num_valid.value_counts()
state_test_df[state_test_df.math_test_pct_prof_low == '-3'].math_test_num_valid.value_counts()

1    282968
2    171436
3    123489
4     96568
5     80653
Name: math_test_num_valid, dtype: int64

In [56]:
# No missing/not reported data (-1)
state_test_df[state_test_df.read_test_pct_prof_low == '-1']

# Some NA data, though (-2)
state_test_df[state_test_df.read_test_pct_prof_low == '-2']

# Seems to happen when there is no one in that demographic slice that took the test
state_test_df[state_test_df.read_test_pct_prof_low == '-2'].read_test_num_valid.value_counts()
state_test_df[state_test_df.math_test_pct_prof_low == '-2'].math_test_num_valid.value_counts()

# But we can't just drop those rows of data YET, because they may have ppl in that slice that
# Didn't take the reading test, but took the math test
# And vice versa
state_test_df[state_test_df.math_test_pct_prof_low == '-2'].read_test_num_valid.value_counts()

0     106687
1        176
2         39
3         20
29         3
9          2
6          1
11         1
12         1
8          1
15         1
18         1
5          1
4          1
22         1
14         1
7          1
Name: read_test_num_valid, dtype: int64

#### Converting row-based data into column-based data

In [57]:
# Creating a comprehensive demographic "slice" column
state_test_df['full_demographics'] =\
state_test_df['grade_edfacts'].astype(str)+\
'_'+state_test_df['sex'].astype(str) +'_'+state_test_df['race'].astype(str)+\
'_'+state_test_df['disability'].astype(str)+'_'+state_test_df['lep'].astype(str)+\
'_'+state_test_df['homeless'].astype(str)+'_'+state_test_df['migrant'].astype(str)+\
'_'+state_test_df['econ_disadvantaged'].astype(str)+'_'+state_test_df['foster_care'].astype(str)+\
'_'+state_test_df['military_connected'].astype(str)

In [58]:
# This cell shows that there's only one row per school that represents 'entire school'
max(state_test_df[state_test_df.full_demographics == '99_99_99_99_99_99_99_99_99_99'].groupby('ncessch').year.count())

1

In [59]:
# This cell shows us that each school's data starts with the entire school data
state_test_df[['ncessch', 'full_demographics']].groupby('ncessch').first().full_demographics.value_counts()

99_99_99_99_99_99_99_99_99_99    86245
Name: full_demographics, dtype: int64

In [60]:
# Note there are 128 combinations of demographic info
state_test_df.full_demographics.nunique() 
state_test_df.full_demographics.value_counts()

99_99_99_99_99_99_99_99_99_99    86245
99_1_99_99_99_99_99_99_99_99     85894
99_2_99_99_99_99_99_99_99_99     85520
99_99_99_99_99_99_99_1_99_99     85005
99_99_1_99_99_99_99_99_99_99     82444
                                 ...  
9_99_4_99_99_99_99_99_99_99      11878
6_99_5_99_99_99_99_99_99_99      11374
8_99_5_99_99_99_99_99_99_99      10496
7_99_5_99_99_99_99_99_99_99      10394
9_99_5_99_99_99_99_99_99_99       8374
Name: full_demographics, Length: 88, dtype: int64

In [61]:
# Creating a non-comprehensive, selective demographic "slice" column
# This will make the dataframe more manageable
# We only chose grade, sex, race, disability, lep, homeless, and economically disadvantaged, 
# beacuse this is similar to the demographic data supplied by the CRDC SAT ACT data
state_test_df['grade_sex_race_disability_lep_disadv'] =\
state_test_df['grade_edfacts'].astype(str)+'_'+state_test_df['sex'].astype(str) +'_'+state_test_df['race'].astype(str)+\
'_'+state_test_df['disability'].astype(str)+'_'+state_test_df['lep'].astype(str)+\
'_'+state_test_df['econ_disadvantaged'].astype(str)

In [62]:
state_test_df['grade_sex_race_disability_lep_disadv'] = state_test_df['grade_sex_race_disability_lep_disadv'].astype('category')

##### Splitting data into math and reading test data, because there are simply too many columns

In [63]:
# Only kept the proficiency share midpoint values
# Make a helpful copy with only a select number of columns
read_state_test_df = state_test_df[['ncessch', 'school_name', 'leaid_num', 'leaid',
       'lea_name', 'fips', 'read_test_num_valid','read_test_pct_prof_midpt',
        'grade_sex_race_disability_lep_disadv','full_demographics']].copy('deep')

math_state_test_df = state_test_df[['ncessch', 'school_name', 'leaid_num', 'leaid',
       'lea_name', 'fips', 'math_test_num_valid', 'math_test_pct_prof_midpt',
        'grade_sex_race_disability_lep_disadv','full_demographics']].copy('deep')

In [64]:
# Drop data with special value -2 (i.e. NA) and -3 (suppressed)
read_state_test_df = read_state_test_df[read_state_test_df.read_test_pct_prof_midpt != '-2.0']
read_state_test_df = read_state_test_df[read_state_test_df.read_test_pct_prof_midpt != '-3.0']

math_state_test_df = math_state_test_df[math_state_test_df.math_test_pct_prof_midpt != '-2.0']
math_state_test_df = math_state_test_df[math_state_test_df.math_test_pct_prof_midpt != '-3.0']

In [65]:
# Get dummies of the demographic slice data
read_state_test_dem_df = pd.get_dummies(read_state_test_df, columns = ['grade_sex_race_disability_lep_disadv'])
math_state_test_dem_df = pd.get_dummies(math_state_test_df, columns = ['grade_sex_race_disability_lep_disadv'])

In [66]:
dummy_cols = [c for c in read_state_test_dem_df.columns if 'grade' in c]

In [67]:
# Multiply out the proficiency score with the dummy columns
for c in dummy_cols:
    read_state_test_dem_df[c] = np.where(read_state_test_dem_df[c] == 1,\
                                         read_state_test_dem_df[c] * read_state_test_dem_df.read_test_pct_prof_midpt, 0)
    
for c in dummy_cols:
    math_state_test_dem_df[c] = np.where(math_state_test_dem_df[c] == 1,\
                                         math_state_test_dem_df[c] * math_state_test_dem_df.math_test_pct_prof_midpt, 0)

In [68]:
# Create dictionaries with information on how to aggregate the row-based data
read_agg_test_dict = {'fips':'first','school_name':'first',
                      'read_test_num_valid':'first'}
read_agg_test_dict.update(dict(zip(dummy_cols,['sum']*len(dummy_cols))))
math_agg_test_dict = {'fips':'first','school_name':'first',
                      'math_test_num_valid':'first'}
math_agg_test_dict.update(dict(zip(dummy_cols,['sum']*len(dummy_cols))))

In [69]:
# Cast to correct types
for c in dummy_cols:
    read_state_test_dem_df[c] = read_state_test_dem_df[c].astype(float)
    math_state_test_dem_df[c] = math_state_test_dem_df[c].astype(float)

In [70]:
# Perform aggregation on the reading df
grouped_read_df = read_state_test_dem_df.groupby('ncessch').agg(read_agg_test_dict)
grouped_read_df.reset_index(inplace=True)
grouped_read_df.head()

,ncessch,fips,school_name,read_test_num_valid,grade_sex_race_disability_lep_disadv_3_1_99_99_99_99,grade_sex_race_disability_lep_disadv_3_2_99_99_99_99,grade_sex_race_disability_lep_disadv_3_99_1_99_99_99,grade_sex_race_disability_lep_disadv_3_99_2_99_99_99,grade_sex_race_disability_lep_disadv_3_99_3_99_99_99,grade_sex_race_disability_lep_disadv_3_99_4_99_99_99,grade_sex_race_disability_lep_disadv_3_99_5_99_99_99,grade_sex_race_disability_lep_disadv_3_99_7_99_99_99,grade_sex_race_disability_lep_disadv_3_99_99_99_1_99,grade_sex_race_disability_lep_disadv_3_99_99_99_99_1,grade_sex_race_disability_lep_disadv_3_99_99_99_99_99,grade_sex_race_disability_lep_disadv_4_1_99_99_99_99,grade_sex_race_disability_lep_disadv_4_2_99_99_99_99,grade_sex_race_disability_lep_disadv_4_99_1_99_99_99,grade_sex_race_disability_lep_disadv_4_99_2_99_99_99,grade_sex_race_disability_lep_disadv_4_99_3_99_99_99,grade_sex_race_disability_lep_disadv_4_99_4_99_99_99,grade_sex_race_disability_lep_disadv_4_99_5_99_99_99,grade_sex_race_disability_lep_disadv_4_99_7_99_99_99,grade_sex_race_disability_lep_disadv_4_99_99_99_1_99,grade_sex_race_disability_lep_disadv_4_99_99_99_99_1,grade_sex_race_disability_lep_disadv_4_99_99_99_99_99,grade_sex_race_disability_lep_disadv_5_1_99_99_99_99,grade_sex_race_disability_lep_disadv_5_2_99_99_99_99,grade_sex_race_disability_lep_disadv_5_99_1_99_99_99,grade_sex_race_disability_lep_disadv_5_99_2_99_99_99,grade_sex_race_disability_lep_disadv_5_99_3_99_99_99,grade_sex_race_disability_lep_disadv_5_99_4_99_99_99,grade_sex_race_disability_lep_disadv_5_99_5_99_99_99,grade_sex_race_disability_lep_disadv_5_99_7_99_99_99,grade_sex_race_disability_lep_disadv_5_99_99_99_1_99,grade_sex_race_disability_lep_disadv_5_99_99_99_99_1,grade_sex_race_disability_lep_disadv_5_99_99_99_99_99,grade_sex_race_disability_lep_disadv_6_1_99_99_99_99,grade_sex_race_disability_lep_disadv_6_2_99_99_99_99,grade_sex_race_disability_lep_disadv_6_99_1_99_99_99,grade_sex_race_disability_lep_disadv_6_99_2_99_99_99,grade_sex_race_disability_lep_disadv_6_99_3_99_99_99,grade_sex_race_disability_lep_disadv_6_99_4_99_99_99,grade_sex_race_disability_lep_disadv_6_99_5_99_99_99,grade_sex_race_disability_lep_disadv_6_99_7_99_99_99,grade_sex_race_disability_lep_disadv_6_99_99_99_1_99,grade_sex_race_disability_lep_disadv_6_99_99_99_99_1,grade_sex_race_disability_lep_disadv_6_99_99_99_99_99,grade_sex_race_disability_lep_disadv_7_1_99_99_99_99,grade_sex_race_disability_lep_disadv_7_2_99_99_99_99,grade_sex_race_disability_lep_disadv_7_99_1_99_99_99,grade_sex_race_disability_lep_disadv_7_99_2_99_99_99,grade_sex_race_disability_lep_disadv_7_99_3_99_99_99,grade_sex_race_disability_lep_disadv_7_99_4_99_99_99,grade_sex_race_disability_lep_disadv_7_99_5_99_99_99,grade_sex_race_disability_lep_disadv_7_99_7_99_99_99,grade_sex_race_disability_lep_disadv_7_99_99_99_1_99,grade_sex_race_disability_lep_disadv_7_99_99_99_99_1,grade_sex_race_disability_lep_disadv_7_99_99_99_99_99,grade_sex_race_disability_lep_disadv_8_1_99_99_99_99,grade_sex_race_disability_lep_disadv_8_2_99_99_99_99,grade_sex_race_disability_lep_disadv_8_99_1_99_99_99,grade_sex_race_disability_lep_disadv_8_99_2_99_99_99,grade_sex_race_disability_lep_disadv_8_99_3_99_99_99,grade_sex_race_disability_lep_disadv_8_99_4_99_99_99,grade_sex_race_disability_lep_disadv_8_99_5_99_99_99,grade_sex_race_disability_lep_disadv_8_99_7_99_99_99,grade_sex_race_disability_lep_disadv_8_99_99_99_1_99,grade_sex_race_disability_lep_disadv_8_99_99_99_99_1,grade_sex_race_disability_lep_disadv_8_99_99_99_99_99,grade_sex_race_disability_lep_disadv_99_1_99_99_99_99,grade_sex_race_disability_lep_disadv_99_2_99_99_99_99,grade_sex_race_disability_lep_disadv_99_99_1_99_99_99,grade_sex_race_disability_lep_disadv_99_99_2_99_99_99,grade_sex_race_disability_lep_disadv_99_99_3_99_99_99,grade_sex_race_disability_lep_disadv_99_99_4_99_99_99,grade_sex_race_disability_lep_disadv_99_99_5_99_99_99,grade_sex_race_disability_lep_disadv_99_99_7_99_99_99,grade_sex_race_dis

In [71]:
# Perform aggregation on the math df
grouped_math_df = math_state_test_dem_df.groupby('ncessch').agg(math_agg_test_dict)
grouped_math_df.reset_index(inplace=True)
grouped_math_df.head()

,ncessch,fips,school_name,math_test_num_valid,grade_sex_race_disability_lep_disadv_3_1_99_99_99_99,grade_sex_race_disability_lep_disadv_3_2_99_99_99_99,grade_sex_race_disability_lep_disadv_3_99_1_99_99_99,grade_sex_race_disability_lep_disadv_3_99_2_99_99_99,grade_sex_race_disability_lep_disadv_3_99_3_99_99_99,grade_sex_race_disability_lep_disadv_3_99_4_99_99_99,grade_sex_race_disability_lep_disadv_3_99_5_99_99_99,grade_sex_race_disability_lep_disadv_3_99_7_99_99_99,grade_sex_race_disability_lep_disadv_3_99_99_99_1_99,grade_sex_race_disability_lep_disadv_3_99_99_99_99_1,grade_sex_race_disability_lep_disadv_3_99_99_99_99_99,grade_sex_race_disability_lep_disadv_4_1_99_99_99_99,grade_sex_race_disability_lep_disadv_4_2_99_99_99_99,grade_sex_race_disability_lep_disadv_4_99_1_99_99_99,grade_sex_race_disability_lep_disadv_4_99_2_99_99_99,grade_sex_race_disability_lep_disadv_4_99_3_99_99_99,grade_sex_race_disability_lep_disadv_4_99_4_99_99_99,grade_sex_race_disability_lep_disadv_4_99_5_99_99_99,grade_sex_race_disability_lep_disadv_4_99_7_99_99_99,grade_sex_race_disability_lep_disadv_4_99_99_99_1_99,grade_sex_race_disability_lep_disadv_4_99_99_99_99_1,grade_sex_race_disability_lep_disadv_4_99_99_99_99_99,grade_sex_race_disability_lep_disadv_5_1_99_99_99_99,grade_sex_race_disability_lep_disadv_5_2_99_99_99_99,grade_sex_race_disability_lep_disadv_5_99_1_99_99_99,grade_sex_race_disability_lep_disadv_5_99_2_99_99_99,grade_sex_race_disability_lep_disadv_5_99_3_99_99_99,grade_sex_race_disability_lep_disadv_5_99_4_99_99_99,grade_sex_race_disability_lep_disadv_5_99_5_99_99_99,grade_sex_race_disability_lep_disadv_5_99_7_99_99_99,grade_sex_race_disability_lep_disadv_5_99_99_99_1_99,grade_sex_race_disability_lep_disadv_5_99_99_99_99_1,grade_sex_race_disability_lep_disadv_5_99_99_99_99_99,grade_sex_race_disability_lep_disadv_6_1_99_99_99_99,grade_sex_race_disability_lep_disadv_6_2_99_99_99_99,grade_sex_race_disability_lep_disadv_6_99_1_99_99_99,grade_sex_race_disability_lep_disadv_6_99_2_99_99_99,grade_sex_race_disability_lep_disadv_6_99_3_99_99_99,grade_sex_race_disability_lep_disadv_6_99_4_99_99_99,grade_sex_race_disability_lep_disadv_6_99_5_99_99_99,grade_sex_race_disability_lep_disadv_6_99_7_99_99_99,grade_sex_race_disability_lep_disadv_6_99_99_99_1_99,grade_sex_race_disability_lep_disadv_6_99_99_99_99_1,grade_sex_race_disability_lep_disadv_6_99_99_99_99_99,grade_sex_race_disability_lep_disadv_7_1_99_99_99_99,grade_sex_race_disability_lep_disadv_7_2_99_99_99_99,grade_sex_race_disability_lep_disadv_7_99_1_99_99_99,grade_sex_race_disability_lep_disadv_7_99_2_99_99_99,grade_sex_race_disability_lep_disadv_7_99_3_99_99_99,grade_sex_race_disability_lep_disadv_7_99_4_99_99_99,grade_sex_race_disability_lep_disadv_7_99_5_99_99_99,grade_sex_race_disability_lep_disadv_7_99_7_99_99_99,grade_sex_race_disability_lep_disadv_7_99_99_99_1_99,grade_sex_race_disability_lep_disadv_7_99_99_99_99_1,grade_sex_race_disability_lep_disadv_7_99_99_99_99_99,grade_sex_race_disability_lep_disadv_8_1_99_99_99_99,grade_sex_race_disability_lep_disadv_8_2_99_99_99_99,grade_sex_race_disability_lep_disadv_8_99_1_99_99_99,grade_sex_race_disability_lep_disadv_8_99_2_99_99_99,grade_sex_race_disability_lep_disadv_8_99_3_99_99_99,grade_sex_race_disability_lep_disadv_8_99_4_99_99_99,grade_sex_race_disability_lep_disadv_8_99_5_99_99_99,grade_sex_race_disability_lep_disadv_8_99_7_99_99_99,grade_sex_race_disability_lep_disadv_8_99_99_99_1_99,grade_sex_race_disability_lep_disadv_8_99_99_99_99_1,grade_sex_race_disability_lep_disadv_8_99_99_99_99_99,grade_sex_race_disability_lep_disadv_99_1_99_99_99_99,grade_sex_race_disability_lep_disadv_99_2_99_99_99_99,grade_sex_race_disability_lep_disadv_99_99_1_99_99_99,grade_sex_race_disability_lep_disadv_99_99_2_99_99_99,grade_sex_race_disability_lep_disadv_99_99_3_99_99_99,grade_sex_race_disability_lep_disadv_99_99_4_99_99_99,grade_sex_race_disability_lep_disadv_99_99_5_99_99_99,grade_sex_race_disability_lep_disadv_99_99_7_99_99_99,grade_sex_race_dis

### Exporting Cleaned Data

In [72]:
# ccd_directory_df.to_csv('clean_ccd_directory.csv')
# dask_sat_act_df.compute().to_csv('clean_sat_act.csv')
grouped_read_df.to_csv('clean_read.csv')
grouped_math_df.to_csv('clean_math.csv')